In [ ]:
#hands pn lab
"""You are a Data Engineer tasked with building a Q1 2024 sales analytics pipeline. You receive partitioned monthly 
data that must be integrated with dimension tables, analyzed for KPIs, and optimized for production BI consumption.
Lab Objectives
Concatenate partitioned datasets with hierarchical indexing
Perform complex multi-table joins with overlapping columns
Execute advanced GroupBy operations (filter, transform, apply)
Generate executive pivot tables with margins
Implement time series resampling and rolling windows
Optimize memory using categorical data types and proper indexing
"""
import pandas as pd
from glob import glob

# Step 1: Load partitioned datasets
file_paths = glob("data/sales_2024_*.csv")
sales_data = pd.concat([pd.read_csv(f) for f in file_paths], ignore_index=True)

# Step 2: Process Date for Hierarchical Indexing
sales_data['Date'] = pd.to_datetime(sales_data['Date'])
sales_data['Year'] = sales_data['Date'].dt.year
sales_data['Month'] = sales_data['Date'].dt.month
sales_data.set_index(['Year', 'Month'], inplace=True)


# Step 4: Merge with Dimension Tables
sales_data = sales_data.merge(products, on='ProductID', how='left')
sales_data = sales_data.merge(customers, on='CustomerID', how='left')

# Step 5: Rename Overlapping Columns if necessary
sales_data = sales_data.rename(columns={
    'SalesAmount_x': 'SalesAmount', 
    'SalesAmount_y': 'RefundAmount'
})

# Step 6: Execute Complex GroupBy Operations
kpis = sales_data.groupby(['ProductID', 'CustomerID']).agg(
    TotalSales=('SalesAmount', 'sum'),
    AvgDiscount=('Discount', 'mean')
).reset_index()


# Step 7: Create Executive Pivot Table
pivot_table = sales_data.pivot_table(
    index=['ProductCategory'],
    values='SalesAmount',
    aggfunc='sum',
    margins=True,
    margins_name='Total'
)

# Step 8: Time Series Resampling
sales_data.set_index('Date', inplace=True)
daily_sales = sales_data.resample('D').sum()

# Step 9: Implement Rolling Windows Analysis
daily_sales['RollingAvg'] = daily_sales['SalesAmount'].rolling(window=7).mean()


# Step 10: Optimize Memory Usage with Categorical Types
sales_data['ProductID'] = sales_data['ProductID'].astype('category')
sales_data['CustomerID'] = sales_data['CustomerID'].astype('category')
sales_data.set_index(['ProductID', 'CustomerID'], inplace=True)

# Step 11: Export Final DataFrame for BI Tools
sales_data.to_csv("data/optimized_sales_data.csv", index=False)


#assignment 1
#Analyze sales data with advanced grouping and pivot tables.

In [16]:
import pandas as pd
df=pd.read_csv("Sales-Export_2019-2020.csv")
df.head()

,country,order_value_EUR,cost,date,category,customer_name,sales_manager,sales_rep,device_type,order_id
0,Sweden,"17,524.02",14122.61,2/12/2020,Books,Goldner-Dibbert,Maxie Marrow,Madelon Bront,Mobile,70-0511466
1,Finland,"116,563.40",92807.78,9/26/2019,Games,Hilll-Vandervort,Hube Corey,Wat Bowkley,Mobile,28-6585323
2,Portugal,"296,465.56",257480.34,7/11/2019,Clothing,Larkin-Collier,Celine Tumasian,Smitty Culverhouse,PC,58-7703341
3,Portugal,"74,532.02",59752.32,4/2/2020,Beauty,Hessel-Stiedemann,Celine Tumasian,Aurelie Wren,PC,14-6700183
4,Spain,"178,763.42",146621.76,12/22/2019,Games,Johns and Sons,Emalia Dinse,Bertha Walbrook,Tablet,15-8765160


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   country            1000 non-null   object 
 1    order_value_EUR   1000 non-null   object 
 2    cost              1000 non-null   float64
 3   date               1000 non-null   object 
 4   category           1000 non-null   object 
 5   customer_name      1000 non-null   object 
 6   sales_manager      1000 non-null   object 
 7   sales_rep          1000 non-null   object 
 8   device_type        1000 non-null   object 
 9   order_id           1000 non-null   object 
dtypes: float64(1), object(9)
memory usage: 78.3+ KB


In [18]:
# clean white space from column names
df.columns = df.columns.str.strip()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   country          1000 non-null   object 
 1   order_value_EUR  1000 non-null   object 
 2   cost             1000 non-null   float64
 3   date             1000 non-null   object 
 4   category         1000 non-null   object 
 5   customer_name    1000 non-null   object 
 6   sales_manager    1000 non-null   object 
 7   sales_rep        1000 non-null   object 
 8   device_type      1000 non-null   object 
 9   order_id         1000 non-null   object 
dtypes: float64(1), object(9)
memory usage: 78.3+ KB


In [19]:
df['date'] = pd.to_datetime(df['date'])

In [20]:
df["order_value_EUR"] = df["order_value_EUR"].str.replace(",", "").astype(float)


In [22]:
df['profit']= (df['order_value_EUR'] - df['cost']).round(2)
df.head()

,country,order_value_EUR,cost,date,category,customer_name,sales_manager,sales_rep,device_type,order_id,profit
0,Sweden,17524.02,14122.61,2020-02-12,Books,Goldner-Dibbert,Maxie Marrow,Madelon Bront,Mobile,70-0511466,3401.41
1,Finland,116563.40,92807.78,2019-09-26,Games,Hilll-Vandervort,Hube Corey,Wat Bowkley,Mobile,28-6585323,23755.62
2,Portugal,296465.56,257480.34,2019-07-11,Clothing,Larkin-Collier,Celine Tumasian,Smitty Culverhouse,PC,58-7703341,38985.22
3,Portugal,74532.02,59752.32,2020-04-02,Beauty,Hessel-Stiedemann,Celine Tumasian,Aurelie Wren,PC,14-6700183,14779.70
4,Spain,178763.42,146621.76,2019-12-22,Games,Johns and Sons,Emalia Dinse,Bertha Walbrook,Tablet,15-8765160,32141.66


In [24]:
df.describe()  

,order_value_EUR,cost,date,profit
count,1000.000000,1000.000000,1000,1000.000000
mean,113361.738710,94369.310990,2020-01-01 05:41:16.799999744,18992.427720
min,15100.570000,12113.680000,2019-01-02 00:00:00,2073.610000
25%,65310.862500,54247.957500,2019-06-30 18:00:00,10746.402500
50%,105419.425000,87094.760000,2020-01-05 00:00:00,17331.830000
75%,151192.642500,125570.837500,2020-06-26 00:00:00,25205.325000
max,383996.760000,304701.430000,2020-12-30 00:00:00,79295.330000
std,61775.335675,51540.020934,NaN,10917.650434


In [ ]:
# optimize memory usage by converting object columns to categorical
categorical_cols = df.select_dtypes(include='object').columns
for col in categorical_cols:
    df[col] = df[col].astype('category')    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   country          1000 non-null   category      
 1   order_value_EUR  1000 non-null   float64       
 2   cost             1000 non-null   float64       
 3   date             1000 non-null   datetime64[ns]
 4   category         1000 non-null   category      
 5   customer_name    1000 non-null   category      
 6   sales_manager    1000 non-null   category      
 7   sales_rep        1000 non-null   category      
 8   device_type      1000 non-null   category      
 9   order_id         1000 non-null   category      
 10  profit           1000 non-null   float64       
dtypes: category(7), datetime64[ns](1), float64(3)
memory usage: 85.1 KB


#assignment 2
#Build a time series analysis for stock market data.